In [71]:
#James Chartouni
from multiprocessing import Pool, cpu_count
import gc; gc.enable()
import pandas as pd
import numpy as np

In [55]:
train = pd.read_csv('raw_data/train.csv')
test = pd.read_csv('raw_data/sample_submission_zero.csv')

print("transactions")
transactions = pd.read_csv('cleaned_input/transactions.csv', usecols=['msno'], dtype={"msno":str})
transactions = pd.DataFrame(transactions['msno'].value_counts().reset_index())
transactions.columns = ['msno','trans_count']
train = pd.merge(train, transactions, how='left', on='msno')
print(train.shape)
test = pd.merge(test, transactions, how='left', on='msno')
transactions = []; print('transaction merge...')

print("user_logs")
user_logs = pd.read_csv("cleaned_input/processed_user_log.csv")
user_logs_2 = pd.read_csv("cleaned_input/processed_user_log_v2.csv")
user_logs = pd.concat((user_logs, user_logs_2))

'''user_logs = pd.read_csv('raw_data/user_logs.csv', usecols=['msno'])
user_logs_2 = pd.read_csv('raw_data/user_logs_v2.csv', usecols=['msno'])
user_logs = pd.concat((user_logs, user_logs_2))
user_logs = pd.DataFrame(user_logs['msno'].value_counts().reset_index())
#user_logs.to_csv('cleaned_input/user_logs_merged.csv')
user_logs_reduced = user_logs
user_logs_reduced.columns = ['msno','logs_count']'''

print(train.shape)
print(user_logs.shape)
train = pd.merge(train, user_logs, how='left', on='msno')
test = pd.merge(test, user_logs, how='left', on='msno')
user_logs_reduced = []

print("members")
members = pd.read_csv('cleaned_input/members.csv')
train = pd.merge(train, members, how='left', on='msno')
test = pd.merge(test, members, how='left', on='msno')
members = []; print('members merge...') 

transactions
(992931, 3)
transaction merge...
user_logs
(992931, 3)
(6338005, 11)
members
members merge...


In [56]:
#combine user_log_time_series
user_log_time_series = pd.read_csv("100_user_logs_features.csv")
user_log_time_series_1 = pd.read_csv("100_user_logs_features.csv")
user_log_time_series_2 = pd.read_csv("200_user_logs_features.csv")
user_log_time_series_3 = pd.read_csv("300_user_logs_features.csv")
user_log_time_series_4 = pd.read_csv("400_user_logs_features.csv")
user_log_time_series_5 = pd.read_csv("500_user_logs_features.csv")
user_log_time_series_6 = pd.read_csv("600_user_logs_features.csv")
user_log_time_series_7 = pd.read_csv("5100_user_logs_features_3.csv")
user_log_time_series_8 = pd.read_csv("5200_user_logs_features_3.csv")
user_log_time_series_9 = pd.read_csv("5300_user_logs_features_3.csv")

user_log_time_series.append(user_log_time_series_1)
user_log_time_series.append(user_log_time_series_2)
user_log_time_series.append(user_log_time_series_3)
user_log_time_series.append(user_log_time_series_4)
user_log_time_series.append(user_log_time_series_5)
user_log_time_series.append(user_log_time_series_6)
user_log_time_series.append(user_log_time_series_7)
user_log_time_series.append(user_log_time_series_8)
user_log_time_series.append(user_log_time_series_9)

user_log_time_series = user_log_time_series.drop(user_log_time_series.columns[0], axis=1)

In [57]:
#user_log_time_series = pd.to_numeric(user_log_time_series[user_log_time_series.columns[1]])
user_log_time_series["gap_days"] = user_log_time_series["gap_days"].astype(str).str.split().str[0].str.rsplit('.').str[0]
user_log_time_series.head()
#.astype(np.int8)

,msno,gap_days,average_gap_days,longest_gap,gaps_longer_29
0,yxiEWwE9VR5utpUecLxVdQ5B7NysUPfrNtGINaM2zA8=,732,5.270400e+06,7430400.0,12.0
1,PNxIsSLWOJDCm7pNPFzRO/6Mmg2WeZA2nf6hw6t1x3g=,402,1.736640e+07,18144000.0,2.0
2,KXF9c/T66LZIzFq+xS64icWMhDQE6miCZAtdXRjZHX8=,532,1.532160e+07,25747200.0,3.0
3,OcZ3fKQzHrL1BuzuFRzUiwPr8l9P8JKCFpafdYf5sCY=,689,5.411782e+06,6652800.0,11.0
4,dW5DQO92XaG024D2QYmktyUnLc2axil74zUPAhF8sEs=,673,5.814720e+06,10627200.0,10.0


In [58]:
user_log_time_series.to_csv("cleaned_input/user_log_time_series.csv")

In [59]:
print(train.shape)
print(user_logs.shape)
print(user_log_time_series.shape)

(1731290, 22)
(6338005, 11)
(131992, 5)


In [60]:
transactions = pd.read_csv('raw_data/transactions.csv')
transactions = transactions.sort_values(by=['transaction_date'], ascending=[False]).reset_index(drop=True)
transactions = transactions.drop_duplicates(subset=['msno'], keep='first')

train = pd.merge(train, transactions, how='left', on='msno')
test = pd.merge(test, transactions, how='left', on='msno')
transactions=[]

In [61]:
train.columns

Index(['msno', 'is_churn', 'trans_count', 'date_min', 'date_max', 'date_count',
       'num_25_sum', 'num_50_sum', 'num_75_sum', 'num_985_sum', 'num_100_sum',
       'num_unq_sum', 'total_secs_sum', 'Unnamed: 0', 'city', 'bd', 'gender',
       'registered_via', 'registration_init_time', 'registration_init_year',
       'registration_init_month', 'registration_init_date',
       'payment_method_id', 'payment_plan_days', 'plan_list_price',
       'actual_amount_paid', 'is_auto_renew', 'transaction_date',
       'membership_expire_date', 'is_cancel'],
      dtype='object')

In [62]:
'''def transform_df(df):
    df = pd.DataFrame(df)
    df = df.sort_values(by=['date'], ascending=[False])
    df = df.reset_index(drop=True)
    df = df.drop_duplicates(subset=['msno'], keep='first')
    return df

def transform_df2(df):
    df = df.sort_values(by=['date'], ascending=[False])
    df = df.reset_index(drop=True)
    df = df.drop_duplicates(subset=['msno'], keep='first')
    return df

#make sure you add user_logs_2
df_iter = pd.read_csv('raw_data/user_logs.csv', low_memory=False, iterator=True, chunksize=10000000)
last_user_logs = []
i = 0 #~400 Million Records - starting at the end but remove locally if needed
for df in df_iter:
    if i>35:
        if len(df)>0:
            print(df.shape)
            p = Pool(cpu_count())
            df = p.map(transform_df, np.array_split(df, cpu_count()))   
            df = pd.concat(df, axis=0, ignore_index=True).reset_index(drop=True)
            df = transform_df2(df)
            p.close(); p.join()
            last_user_logs.append(df)
            print('...', df.shape)
            df = []
    i+=1

last_user_logs = pd.concat(last_user_logs, axis=0, ignore_index=True).reset_index(drop=True)
last_user_logs = transform_df2(last_user_logs)

train = pd.merge(train, last_user_logs, how='left', on='msno')
test = pd.merge(test, last_user_logs, how='left', on='msno')
last_user_logs=[]'''

"def transform_df(df):\n    df = pd.DataFrame(df)\n    df = df.sort_values(by=['date'], ascending=[False])\n    df = df.reset_index(drop=True)\n    df = df.drop_duplicates(subset=['msno'], keep='first')\n    return df\n\ndef transform_df2(df):\n    df = df.sort_values(by=['date'], ascending=[False])\n    df = df.reset_index(drop=True)\n    df = df.drop_duplicates(subset=['msno'], keep='first')\n    return df\n\n#make sure you add user_logs_2\ndf_iter = pd.read_csv('raw_data/user_logs.csv', low_memory=False, iterator=True, chunksize=10000000)\nlast_user_logs = []\ni = 0 #~400 Million Records - starting at the end but remove locally if needed\nfor df in df_iter:\n    if i>35:\n        if len(df)>0:\n            print(df.shape)\n            p = Pool(cpu_count())\n            df = p.map(transform_df, np.array_split(df, cpu_count()))   \n            df = pd.concat(df, axis=0, ignore_index=True).reset_index(drop=True)\n            df = transform_df2(df)\n            p.close(); p.join()\n    

In [63]:
#if missing >3 cells, remove row 
train = train.dropna(thresh=3)
test = test.dropna(thresh=3)

In [64]:
#remove outliers
train = train.drop(train[train.bd < 7].index)
train = train.drop(train[train.bd > 99].index)
test = test.drop(test[test.bd < 7].index)
test = test.drop(test[test.bd > 99].index)

In [65]:
print(train.shape)
print(test.shape)

train.to_csv("cleaned_input/train_merged.csv")
test.to_csv("cleaned_input/test_merged.csv")

(853753, 30)
(857110, 30)


In [96]:
#shortcut so don't have to load the first part 
train = pd.read_csv('cleaned_input/train_merged.csv')
test = pd.read_csv("cleaned_input/test_merged.csv")
#user_logs = pd.read_csv('cleaned_input/user_logs_merged.csv')

In [97]:
data = pd.concat(( train, test ))
train_cutoff = len(train)

In [98]:
data.columns

Index(['Unnamed: 0', 'msno', 'is_churn', 'trans_count', 'date_min', 'date_max',
       'date_count', 'num_25_sum', 'num_50_sum', 'num_75_sum', 'num_985_sum',
       'num_100_sum', 'num_unq_sum', 'total_secs_sum', 'Unnamed: 0.1', 'city',
       'bd', 'gender', 'registered_via', 'registration_init_time',
       'registration_init_year', 'registration_init_month',
       'registration_init_date', 'payment_method_id', 'payment_plan_days',
       'plan_list_price', 'actual_amount_paid', 'is_auto_renew',
       'transaction_date', 'membership_expire_date', 'is_cancel'],
      dtype='object')

In [99]:
#fill in missing data points and encode variables
from helpers import *

nan_zero = {"logs_count":0}
nan_dif = {"city":-1, "registered_via":-1, "payment_method_id":-1, "is_auto_renew":-1, "is_cancel":-1}
nan_med = {"bd": 'med', "registration_init_date": 'med', "registration_init_year":'med', "registration_init_month":'med', 
           "registration_init_date":'med', "registration_init_time":'med', "num_25_sum": 'med', "num_50_sum": 'med', "num_75_sum": 'med', "num_985_sum": 'med', 
           "num_100_sum": 'med', "num_unq_sum": 'med', "total_secs_sum": 'med', "trans_count":'med', 'transaction_date':'med',
          'membership_expire_date':'med', 'plan_list_price':'med', 'actual_amount_paid':'med', 'payment_plan_days':'med'
          ,'date_min':'med', "date_max":'med', 'date_count':'med'}
nan_cat = {"gender":"missing"}

data = data.drop(["Unnamed: 0", "Unnamed: 0.1"], axis = 1)
#data = to_fill_na(data, nan_zero)
data = to_fill_na(data, nan_dif)
data = to_fill_na(data, nan_med)
data = to_fill_na(data, nan_cat)
data.head()


,Unnamed: 0,msno,is_churn,trans_count,date_min,date_max,date_count,num_25_sum,num_50_sum,num_75_sum,...,registration_init_month,registration_init_date,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,0,waLDQMmcOu2jLDaV1ddDkgCrB/jl6sD66Xzs0Vqax1Y=,1,2,20161106.0,20170208.0,6.0,51.0,16.0,19.0,...,4.0,6.0,38,30,149,149,0,20170107,20170206,0
1,1,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,23,20150101.0,20170215.0,10.0,447.0,157.0,121.0,...,4.0,7.0,39,30,149,149,1,20170224,20170321,1
2,2,QA7uiXy8vIbUSPOkCf9RwQ3FsT8jVq2OxDr8zqa7bRQ=,1,23,20170319.0,20170319.0,1.0,0.0,0.0,0.0,...,4.0,7.0,39,30,149,149,1,20170224,20170321,1
3,3,fGwBva6hikQmTJzrbz/2Ezjm5Cth5jZUNvXigKK2AFA=,1,10,20160202.0,20170131.0,10.0,1570.0,950.0,302.0,...,10.0,16.0,39,30,149,149,1,20170112,20170203,1
4,4,mT5V8rEpa+8wuqi6x0DoVd3H5icMKkE9Prt49UlmK+4=,1,3,20150101.0,20170221.0,10.0,7265.0,2420.0,1789.0,...,11.0,2.0,38,410,1788,1788,0,20160106,20170220,0


In [ ]:
for col in data:
    if data.isnull().any()
    data = data.drop([col])

In [100]:
data.isnull().any()

msno                       False
is_churn                   False
trans_count                False
date_min                   False
date_max                   False
date_count                 False
num_25_sum                 False
num_50_sum                 False
num_75_sum                 False
num_985_sum                False
num_100_sum                False
num_unq_sum                False
total_secs_sum             False
city                       False
bd                         False
gender                     False
registered_via             False
registration_init_time     False
registration_init_year     False
registration_init_month    False
registration_init_date     False
payment_method_id          False
payment_plan_days          False
plan_list_price            False
actual_amount_paid         False
is_auto_renew              False
transaction_date           False
membership_expire_date     False
is_cancel                  False
dtype: bool

In [102]:
#feature engineering #DONT FORGET TO REMOVE INFINITE IF DIVIDE BY ZERO

data['discount'] = data['plan_list_price'] - data['actual_amount_paid']
data['is_discount'] = data.discount.apply(lambda x: 1 if x > 0 else 0)
data['amt_per_day'] = data['actual_amount_paid'] / data['payment_plan_days']
date_cols = ['transaction_date', 'membership_expire_date']
for col in date_cols:
    data[col] = pd.to_datetime(data[col], format='%Y%m%d')
    
#--- difference in days ---
data['membership_duration'] = data.membership_expire_date - data.transaction_date
data['membership_duration'] = data['membership_duration'] / np.timedelta64(1, 'D')
data = to_fill_na(data, {'membership_duration':'med'})
data['membership_duration'] = data['membership_duration'].astype(int)

#---difference in months ---
data['membership_duration_M'] = (data.membership_expire_date - data.transaction_date)/ np.timedelta64(1, 'M')
data['membership_duration_M'] = round(data['membership_duration_M']).astype(int)
data['membership_duration_M'].head()

date_cols = ['registration_init_time']

for col in date_cols:
    data[col] = pd.to_datetime(data[col], format='%Y%m%d')

'''#--- difference in days ---
data['registration_duration'] = data.expiration_date - data.registration_init_time
data['registration_duration'] = data['registration_duration'] / np.timedelta64(1, 'D')
data = to_fill_na(data, {'registration_duration':'med'})
data['registration_duration'] = data['registration_duration'].astype(int)'''

'''
#---difference in months ---
data['registration_duration_M'] = (data.expiration_date - data.registration_init_time)/ np.timedelta64(1, 'M')
data = to_fill_na(data, {'registration_duration_M':'med'})
data['registration_duration_M'] = round(data['registration_duration_M']).astype(int)

data['reg_mem_duration'] = data['registration_duration'] - data['membership_duration']
to_fill_na(data, {'reg_mem_duration':'med'})
data['reg_mem_duration_M'] = data['registration_duration_M'] - data['membership_duration_M']
'''
data['notAutorenew_&_cancel'] = ((data.is_auto_renew == 0) == (data.is_cancel == 1)).astype(np.int8)
data['notAutorenew_&_cancel'].unique()

#data['long_time_user'] = (((data['registration_duration'] / 365).astype(int)) > 1).astype(int)

#feature ideas to implement
#- price paid/ duration 
data['price_paid/duration'] = data["actual_amount_paid"]/ data["membership_duration"]
#- list price/ duration 
data['list_price/duration'] = data['plan_list_price']/ data["membership_duration"]
#- number of unique songs/ price paid
data["unique_songs/price_paid"] = data['num_unq_sum']/ data['actual_amount_paid']
#- number of unique songs/ list price
data["unique_songs/list_price"] = data['num_unq_sum']/ data['plan_list_price']
#-num25, num50 etc.../ price paid & list price
data["num_25/price_paid"] = data['num_25_sum']/data['actual_amount_paid']
data["num_50/price_paid"] = data['num_50_sum']/data['actual_amount_paid']
data["num_75/price_paid"] = data['num_75_sum']/data['actual_amount_paid']
data["num_985/price_paid"] = data['num_985_sum']/data['actual_amount_paid']
data["num_100/price_paid"] = data['num_100_sum']/data['actual_amount_paid']
data["num_25/list_price"] = data['num_25_sum']/data['plan_list_price']
data["num_50/list_price"] = data['num_50_sum']/data['plan_list_price']
data["num_75/list_price"] = data['num_75_sum']/data['plan_list_price']
data["num_985/list_price"] = data['num_985_sum']/data['plan_list_price']
data["num_100/list_price"] = data['num_100_sum']/data['plan_list_price']
# discount percentage (list price - price paid)/ list price 
data["discount_percentage"] = data['discount']/data['plan_list_price']
# days to expiration (expiration date - date)
#data['days_to_expiration'] = data['payment_plan_days'] - data['registration_duration']
# see if a single user has a pattern of canceling, resubscribing etc...?
# could make some really interesting features like how many times did subscribe:?
# times canceled ?
# Total subscription time  ?
# time in between subscriptions? 
# time in between subscriptions/total subscription time?

# try somes features related to age: 
# price paid/ age 
data['price_paid/age'] = data['actual_amount_paid']/ data['bd']
# unique songs/ age
data['unique_song/age'] = data["num_unq_sum"] / data['bd']
# discount percentage/ age
data['discount_percentage/age'] = data['discount_percentage']/ data['bd']
# days to expiration / age 
#data['days_to_expiration/age'] = data['days_to_expiration'] / data['bd']
# age/ date figures 




In [104]:
from helpers import *

user_logs = [] 

#change to dummy variable 
categorical_to_encode = ['gender']
data = label_encoder(data, categorical_to_encode)

categorical_to_binarizer = ["city", "gender", "registered_via", "payment_method_id", "is_auto_renew", "is_cancel"]
data = label_binarizer(data,categorical_to_binarizer)

variables_to_drop = ["registration_init_time", "transaction_date", "membership_expire_date"] 
data = data.drop(categorical_to_encode, axis=1)
data = data.drop(variables_to_drop, axis=1)

labels: 
['female' 'male' 'missing']


In [105]:
data.dtypes

msno                            object
is_churn                         int64
trans_count                      int64
date_min                       float64
date_max                       float64
date_count                     float64
num_25_sum                     float64
num_50_sum                     float64
num_75_sum                     float64
num_985_sum                    float64
num_100_sum                    float64
num_unq_sum                    float64
total_secs_sum                 float64
city                           float64
bd                             float64
registered_via                 float64
registration_init_year         float64
registration_init_month        float64
registration_init_date         float64
payment_method_id              float64
payment_plan_days              float64
plan_list_price                float64
actual_amount_paid             float64
is_auto_renew                  float64
is_cancel                      float64
discount                 

In [106]:
train = data[:train_cutoff]
test = data[train_cutoff:]
print(train.shape)
print(test.shape)

train.to_csv("cleaned_input/train_consolidated.csv")
test.to_csv("cleaned_input/test_consolidated.csv")

(853753, 123)
(857110, 123)
